# Домашнее задание № 10

# Поиск выбросов и генерация новых признаков

In [1]:
# загрузим базовые библиотеки
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

1. Получение данных

In [2]:
# подключаемся к Glass Classification
import kagglehub

# Download latest version
path = kagglehub.dataset_download("uciml/glass")

print("Path to dataset files:", path)

100%|██████████| 3.42k/3.42k [00:00<00:00, 5.06MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/uciml/glass/versions/1


In [3]:
# считываем данные
path = '/root/.cache/kagglehub/datasets/uciml/glass/versions/1/glass.csv'
df = pd.read_csv(path)
df.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1


Целевая переменная – тип стекла 'Type'. Остальные признаки описывают химические элементы в составе материала.

In [4]:
target = 'Type'

2. Первичный анализ

In [5]:
df.describe()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
count,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000
mean,1.518365,13.407850,2.684533,1.444907,72.650935,0.497056,8.956963,0.175047,0.057009,2.780374
std,0.003037,0.816604,1.442408,0.499270,0.774546,0.652192,1.423153,0.497219,0.097439,2.103739
min,1.511150,10.730000,0.000000,0.290000,69.810000,0.000000,5.430000,0.000000,0.000000,1.000000
25%,1.516522,12.907500,2.115000,1.190000,72.280000,0.122500,8.240000,0.000000,0.000000,1.000000
50%,1.517680,13.300000,3.480000,1.360000,72.790000,0.555000,8.600000,0.000000,0.000000,2.000000
75%,1.519157,13.825000,3.600000,1.630000,73.087500,0.610000,9.172500,0.000000,0.100000,3.000000
max,1.533930,17.380000,4.490000,3.500000,75.410000,6.210000,16.190000,3.150000,0.510000,7.000000


In [7]:
# посмотрим размер
df.shape

(214, 10)

In [8]:
# количество уникальных значений
df.nunique()

,0
RI,178
Na,142
Mg,94
Al,118
Si,133
K,65
Ca,143
Ba,34
Fe,32
Type,6


In [9]:
# проверим на пропуски
df[df.isna().any(axis=1)]

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type


3. Разделим выборку на обучающую и тестовую

In [13]:
# разобъем датасет на обучающую и тестовую выборки
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=target),
                                                    df[target],
                                                    test_size=0.2, random_state=42
                                                    )

Обучим модель на RandomForestClassifier() на не обработанных данных

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

подберем лучшие параметры

In [ ]:
n_samples = df.shape[0]
n_features = df.shape[1]

RFC_param_grid = {'n_estimators': [20,50,100],
                  'max_depth': [None, 2, 5,],
                  'min_samples_split': [2, 0.5, n_samples//2,],
                  'min_samples_leaf': [1, 0.5, n_samples//2,],
                  'max_features': [None, 'sqrt', 'auto', 'log2', 0.3,0.5, n_features//2],
                  'bootstrap':[True, False],
                 }

In [31]:
rf_classifier_grid = GridSearchCV(RandomForestClassifier(random_state=42),
                                  param_grid=RFC_param_grid,
                                  n_jobs=-1,
                                  cv=3,
                                  verbose=1,
                                  refit=True    # для получения лучшей функции
                                 )
rf_classifier_grid.fit(X_train,y_train)

Fitting 3 folds for each of 1134 candidates, totalling 3402 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
486 fits failed out of a total of 3402.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
486 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/uti

GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'bootstrap': [True, False], 'max_depth': [None, 2, 5],
                         'max_features': [None, 'sqrt', 'auto', 'log2', 0.3,
                                          0.5, 5],
                         'min_samples_leaf': [1, 0.5, 107],
                         'min_samples_split': [2, 0.5, 107],
                         'n_estimators': [20, 50, 100]},
             verbose=1)

In [32]:
# посмотрим на оптимальные параметры
rf_classifier_grid.best_params_

{'bootstrap': True,
 'max_depth': None,
 'max_features': 0.3,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 50}

In [34]:
# Получим нашу лучшую модель
model_best_RFC = rf_classifier_grid.best_estimator_
model_best_RFC

RandomForestClassifier(max_features=0.3, n_estimators=50, random_state=42)

5. Получим предсказания и отчет о классификации

In [35]:
# получим предсказания
y_pred = model_best_RFC.predict(X_test)

In [37]:
# вычислим точность модели
model_best_RFC.score(X_test, y_test)

0.8372093023255814

In [38]:
# получим отчет о классификации
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.73      1.00      0.85        11
           2       0.83      0.71      0.77        14
           3       1.00      0.67      0.80         3
           5       1.00      0.50      0.67         4
           6       1.00      1.00      1.00         3
           7       0.89      1.00      0.94         8

    accuracy                           0.84        43
   macro avg       0.91      0.81      0.84        43
weighted avg       0.86      0.84      0.83        43



Ради интереса посмотрим, что нам добавит BaggingClassifier() на не обработанной выборке

In [ ]:
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
BC_param_grid = {
                 'bootstrap': [True, False],
                 'bootstrap_features': [True, False],
                 'n_estimators': [5, 10, 15],
                 'max_samples' : [0.6, 0.8, 1.0],
                 'base_estimator__bootstrap': [True, False],
                 'base_estimator__n_estimators': [100, 200, 300],
                 'base_estimator__max_features' : [0.6, 0.8, 1.0]
             }

In [ ]:
grid_search=GridSearchCV(BaggingClassifier(base_estimator=RandomForestClassifier()), param_grid=param_grid, cv=5)

In [39]:
from sklearn.ensemble import BaggingClassifier
# from sklearn.metrics import accuracy_score, classification_report

# выбираем базовый классификатор
base_classifier = model_best_RFC

# инициализируем bagging-классификатор с базовым классификатором
bagging_classifier = BaggingClassifier(base_classifier, n_estimators=10, random_state=42)

# обучаем
bagging_classifier.fit(X_train, y_train)


BaggingClassifier(estimator=RandomForestClassifier(max_features=0.3,
                                                   n_estimators=50,
                                                   random_state=42),
                  random_state=42)

In [40]:
# получаем предсказания
y_pred = bagging_classifier.predict(X_test)

In [45]:
# вычислим точность модели
model_best_RFC.score(X_test, y_test)

0.8372093023255814

In [46]:
# Расчет точности
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Вывод классификационного отчёта
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.8604651162790697
Classification Report:
              precision    recall  f1-score   support

           1       0.73      1.00      0.85        11
           2       0.92      0.79      0.85        14
           3       1.00      0.67      0.80         3
           5       1.00      0.75      0.86         4
           6       1.00      0.67      0.80         3
           7       0.89      1.00      0.94         8

    accuracy                           0.86        43
   macro avg       0.92      0.81      0.85        43
weighted avg       0.88      0.86      0.86        43

